In [ ]:
#pragma cling(optimize, 3)

#include <string>
#include <fstream>
#include <complex>

#include "xtl/xbase64.hpp"

#include "xtensor/xarray.hpp"
#include "xtensor/xview.hpp"
#include "xtensor/xbuilder.hpp"

#include "xtensor-io/ximage.hpp"

#include "xtensor-fftw/basic.hpp"
#include "xtensor-fftw/helper.hpp"

#include "xwidgets/ximage.hpp"
#include "xwidgets/xbox.hpp"

#include "xwebrtc/xcamera_stream.hpp"
#include "xwebrtc/ximage_recorder.hpp"

In [ ]:
std::vector<char> read_file(const char* filename)
{
    std::basic_ifstream<char> file(filename, std::ios::binary);
    return std::vector<char>((std::istreambuf_iterator<char>(file)), std::istreambuf_iterator<char>());
}

In [ ]:
template <class E>
std::vector<char> to_png_buffer(const xt::xexpression<E>& e)
{
    const char* temp_filename = "/tmp/xio_image.png";
    xt::dump_image(temp_filename, e);
    return read_file(temp_filename);
}

In [ ]:
auto camera_stream = xwebrtc::camera_facing_user(false)
    .constraints({{"audio", false}, {"video", {{"height", 180}, {"width", 320}}}})
    .finalize();
camera_stream

In [ ]:
auto recorder = xwebrtc::image_recorder(camera_stream);
recorder.autosave() = false;
auto& input = recorder.image();
recorder

In [ ]:
recorder.recording = true;

In [ ]:
xt::xarray<double> image_R;
xt::xarray< std::complex<double> > d_image_dx_fs_bw;
xt::xarray< std::complex<double> > d_image_dy_fs_bw;

In [ ]:
auto output = xw::image();

In [ ]:
auto callback = [&](auto&) {
    ::recorder.save();
    
    image_R = xt::view(xt::load_image("record.png"), xt::all(), xt::all(), 0);
    
    auto image_fs_bw = xt::fftw::rfft2(image_R);
    
    std::complex<double> i {0, 1};
    d_image_dx_fs_bw = i * xt::view(xt::fftw::fftscale<double>(image_R.shape()[0]), xt::all(), xt::newaxis()) * image_fs_bw;
    d_image_dy_fs_bw = i * xt::fftw::rfftscale<double>(image_R.shape()[1]) * image_fs_bw;
    
    auto d_image_dx_bw = xt::fftw::irfft2(d_image_dx_fs_bw);
    auto d_image_dy_bw = xt::fftw::irfft2(d_image_dy_fs_bw);
    
    auto d_image_grad_bw = xt::sqrt(d_image_dx_bw * d_image_dx_bw + d_image_dy_bw * d_image_dy_bw);
    
    double amax_d_image_grad_bw3 = xt::amax(d_image_grad_bw)[0];
    
    auto res = xt::cast<unsigned char>(255 - (d_image_grad_bw / amax_d_image_grad_bw3 * 255));
    output.value = to_png_buffer(res);
    
    ::recorder.recording = true;
};
XOBSERVE(input, value, callback);

In [ ]:
recorder.save();

recorder.recording = true;

In [ ]:
output.layout().width = "100%";
output.layout().height = "100%";
input.layout().width = "100%";
input.layout().height = "100%";

In [ ]:
auto hbox = xw::hbox::initialize().children({input, output}).finalize();
hbox